<a href="https://colab.research.google.com/github/piggyatbaqaqi/skol/blob/master/IST691/workspace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SKOL III: Feature extraction

created by:
* La Monte Yarroll
* Padmaja Kurmaddali
* Patrick Le

# **Mapping Data Files: Google Drive**

In [2]:
import os, sys
from pathlib import Path
from google.colab import drive
%cd /content
content = Path('/content')
skol = content / 'drive/My Drive/SKOL'
piggyatbaqaqi = skol / 'github.com/piggyatbaqaqi'
drive.mount(str(content / "drive"), force_remount=True)
cache_path = content / 'cache'
ollama_cache_path = content / 'ollama_cache'
nb_path = content / 'packages'
if not os.path.exists(nb_path):
  nb_path.symlink_to(skol / 'packages')
skol_client = content / 'skol'
if not os.path.exists(skol_client):
  skol_client.symlink_to(piggyatbaqaqi / 'skol')
if not os.path.exists(cache_path):
  cache_path.symlink_to(skol / 'pip_cache')
if not os.path.exists(ollama_cache_path):
  ollama_cache_path.symlink_to(skol / 'ollama_cache')
os.environ['OLLAMA_MODELS'] = str(ollama_cache_path)
sys.path.insert(0, str(nb_path))
sys.path.insert(0, str(piggyatbaqaqi / 'skol'))

/content
Mounted at /content/drive


In [3]:
!ls -l /content/
!file /content
print(sys.path)

total 8
lrwxrwxrwx 1 root root   38 May 19 00:58 cache -> '/content/drive/My Drive/SKOL/pip_cache'
drwx------ 6 root root 4096 May 19 00:58 drive
lrwxrwxrwx 1 root root   41 May 19 00:58 ollama_cache -> '/content/drive/My Drive/SKOL/ollama_cache'
lrwxrwxrwx 1 root root   37 May 19 00:58 packages -> '/content/drive/My Drive/SKOL/packages'
drwxr-xr-x 1 root root 4096 May 14 13:32 sample_data
lrwxrwxrwx 1 root root   58 May 19 00:58 skol -> '/content/drive/My Drive/SKOL/github.com/piggyatbaqaqi/skol'
/content: directory
['/content/drive/My Drive/SKOL/github.com/piggyatbaqaqi/skol', '/content/packages', '/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/root/.ipython', '/usr/local/lib/python3.11/dist-packages/setuptools/_vendor']


## Set up git clients

In [4]:
if not os.path.exists(piggyatbaqaqi):
  %mkdir -p $piggyatbaqaqi
if not os.path.exists(piggyatbaqaqi / 'skol'):
  %cd $piggyatbaqaqi
  !git clone https://github.com/piggyatbaqaqi/skol.git
sys.path.insert(0, piggyatbaqaqi / 'skol')
if not os.path.exists(piggyatbaqaqi / 'dr-drafts-mycosearch'):
  %cd $piggyatbaqaqi
  !git clone https://github.com/piggyatbaqaqi/dr-drafts-mycosearch.git
workdir = skol / 'IST691'
%cd $workdir

/content/drive/.shortcut-targets-by-id/1oRBBGLhTLr9k0-7Qy5rCh8TSd-zhH2mz/SKOL/IST691


In [5]:
# install PySpark
! pip install --cache-dir=$cache_path --target=$nb_path --upgrade pyspark
! pip install --cache-dir=$cache_path --target=$nb_path --upgrade sparknlp ollama

  Using cached pyspark-3.5.5-py2.py3-none-any.whl
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Using cached sparknlp-1.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached spark_nlp-6.0.1-py2.py3-none-any.whl.metadata (19 kB)
  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached sparknlp-1.0.0-py3-none-any.whl (1.4 kB)
Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
Using cached spark_nlp-6.0.1-py2.py3-none-any.whl (697 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 809.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.9/443.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.1 MB/s

In [6]:
#import needed modules
import os
import glob
from typing import List
from pathlib import Path
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.functions import (
    input_file_name, collect_list, concat_ws, col, udf,
    explode, collect_list, regexp_extract, regexp_replace,
    split, flatten, transform, concat)
from pyspark.sql.types import ArrayType, StringType

from pyspark.ml import Pipeline, Transformer
from pyspark.ml.feature import (
    Tokenizer, HashingTF, IDF, StringIndexer, CountVectorizer,
    PCA, VectorAssembler)
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import split, row_number, min, expr, struct
from pyspark.ml.linalg import Vectors, VectorUDT, DenseVector
from pyspark.sql.types import DoubleType, StructField, StructType

SKOL Data manipulation library

In [7]:
from finder import read_files, parse_annotated, target_classes
from label import Label
from taxon import Taxon, group_paragraphs

SEED=12345
default_label = Label('Misc-exposition')
keep_labels = [Label('Description'), Label('Nomenclature')]

# **Checking the file counts in the directories**

In [8]:
raw_directory_path = skol / 'raw_2025_02_05/'
ann_directory_path = skol / 'annotated_2025_02_27/journals'

## Checking the file counts in the directories

In [9]:
# Function that reports all the txt files under a Google Drive folder path
def listFiles(folder: str) -> List[str]:
  # List all files in the folder
  try:
      files = [file for file in glob.glob(f'{folder}/**/*.txt*', recursive=True) if 'Sydowia' not in file]
      return files
  except FileNotFoundError:
      print(f"Folder '{folder}' not found.")
  except PermissionError:
      print(f"Permission denied to access folder '{folder}'.")

In [10]:
# check files in raw directory
listFiles(raw_directory_path)[:10]

['/content/drive/My Drive/SKOL/raw_2025_02_05/authors/FungalNameAuthors.txt',
 '/content/drive/My Drive/SKOL/raw_2025_02_05/Mycologia/3753420.txt',
 '/content/drive/My Drive/SKOL/raw_2025_02_05/Mycologia/3753372.txt',
 '/content/drive/My Drive/SKOL/raw_2025_02_05/Mycologia/3753466.txt',
 '/content/drive/My Drive/SKOL/raw_2025_02_05/Mycologia/3753104.txt',
 '/content/drive/My Drive/SKOL/raw_2025_02_05/Mycologia/3753322.txt',
 '/content/drive/My Drive/SKOL/raw_2025_02_05/Mycologia/mycologiaeuropa00persgoog.txt',
 '/content/drive/My Drive/SKOL/raw_2025_02_05/Mycologia/3753038.txt',
 '/content/drive/My Drive/SKOL/raw_2025_02_05/Mycologia/3753160.txt',
 '/content/drive/My Drive/SKOL/raw_2025_02_05/Mycologia/3753583.txt']

In [12]:
# check files in annotated directory
training_files = listFiles(ann_directory_path)
training_files[:10]

['/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol057/n1.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol054/n1.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s17.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s29.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s30.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s7.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s21.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s13.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s1.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s46.txt.ann']

In [13]:
len(training_files)

190

In [14]:
paragraphs = list(parse_annotated(read_files(random.sample(training_files, 5))))

In [15]:
relabeled = list(target_classes(default=default_label, keep=keep_labels, paragraphs=paragraphs))

In [16]:
print(f'len(relabeled): {len(relabeled)}')

len(relabeled): 702


In [17]:
df = pd.DataFrame({
    'filename': [pp.filename for pp in relabeled],
    'label': [pp.top_label().label if pp.top_label() else None for pp in relabeled],
    'paragraph_number': [pp.paragraph_number for pp in relabeled],
    'page_number': [pp.page_number for pp in relabeled],
    'empirical_page_number': [pp.empirical_page_number for pp in relabeled],
    'line_number': [pp.first_line.line_number if pp.first_line else None for pp in relabeled],
    'body': [str(pp) for pp in relabeled]
})
df.label = pd.Categorical(df.label)
df['label_code'] = df.label.cat.codes

In [18]:
df.groupby('label', observed=True).nunique()

,filename,paragraph_number,page_number,empirical_page_number,line_number,body,label_code
label,,,,,,,
Description,5,19,9,12,15,19,1
Misc-exposition,5,651,18,39,53,449,1
Nomenclature,5,32,8,11,22,32,1


In [19]:
#load in library to open terminal inside google colab
# !pip install  --cache-dir=$cache_path --target=$nb_path colab-xterm
# %load_ext colabxterm


In [24]:
!ls /usr/local/lib/ollama

cuda_v11		  libggml-cpu-haswell.so      libggml-cpu-sse42.so
cuda_v12		  libggml-cpu-icelake.so      libggml-cpu-x64.so
libggml-base.so		  libggml-cpu-sandybridge.so
libggml-cpu-alderlake.so  libggml-cpu-skylakex.so


In [27]:
import subprocess
import time

import ollama
#open up terminal
# %xterm
#once open, first time do:
if not os.path.exists("/usr/local/lib/ollama"):
  !curl https://ollama.ai/install.sh | sh
ollama_server = subprocess.Popen(["ollama", "serve"])
time.sleep(5)  # Let the server come all the way up.
#then start the server with ollama serve &
#first time also will need to pull in a ollama version using ollama pull mistral
# preferred_model 'gema3:12b'
preferred_model 'mistral'
found = False
for _, l in ollama.list():
  for m in l:
    if m.model == preferred_model:
      found = True
      break
if not found:
  !ollama pull $preferred_model


In [30]:
#check here if ollama has a version and can be used, will say model you pulled
!ollama list
!ollama pull mistral

Error: could not connect to ollama app, is it running?
Error: could not connect to ollama app, is it running?


In [29]:
#test if ollama call works
import ollama
response = ollama.chat(model='gemma3:12b', messages=[{
     'role': 'user',
     'content': 'Why is the sky blue?',},
])
print(response['message']['content'])

KeyboardInterrupt: 

In [ ]:
grouped = group_paragraphs(relabeled)
for i, tax in enumerate(grouped):
  if i > 10:
    break
  print(f'Send to LLM:\n\n{tax.as_row()["description"]}')
  #sample message to turn data into json format
  response = ollama.chat(model='mistral', messages=[{
     'role': 'user',
     'content': f'Please turn the following information into a json format: {tax.as_row()}',},
  ])
  print(response['message']['content'])

Send to LLM:

Coloniae eﬀusae, brunneae. Mycelium partim immersum, partim superﬁciale ex hyphis
septatis, ramosis, laevibus, pallide brunneis, 2−4 µm latis, compositum. Conidiophora

solitaria, macronemata, mononemata, recta vel ﬂexuosa, erecta, simplicia, multiseptata,
laevia, brunnea, 135−410 × 4.5−8.5 µm. Cellulae conidiogenae monoblasticae,
terminals, in conidiophoris incorporatae, cum proliferationibus percurrentes, cylindricae,
laeviae, pallide brunneae. Secessio conidiorum rhexolytica. Conidia solitaria, acrogena,
3-euseptata, ellipsoidea, clavata vel pyriformia, simplicia, sicca, laevia, brunnea vel
atrobrunnea, cellula basalis 4.5−6 µm crassa, pallide brunneis, 31.5−42.5 ×14−16.5µm.

Colonies on the natural substrate eﬀuse, brown. Mycelium partly immersed,
partly superﬁcial, composed of septate, branched, smooth-walled, pale
brown hyphae, 2–4 μm thick. Conidiophores solitary, macronematous,
mononematous, straight or ﬂexuous, erect, simple, multiseptate, smooth,
brown, 135–410 